In [1]:
import os
import glob
import psycopg2
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
from geoalchemy2 import Geometry, WKTElement
from shapely.geometry import Point

In [2]:
# Define a database name (we're using a dataset on births, so we'll call it birth_db)
# Set your postgres username
dbname = 'map_the_vote'
username = 'codyschank' # change this to your username

engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)

## create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

postgres://codyschank@localhost/map_the_vote
True


In [3]:
# Connect to make queries using psycopg2
con = None
con = psycopg2.connect(database = dbname, user = username)

In [4]:
path = '/Users/codyschank/Downloads/openaddr-collected-us_south/us/tx/'
extension = 'csv'
os.chdir(path)
files = [i for i in glob.glob('*.{}'.format(extension))]
files = [path + s for s in files]
files = sorted(files)

In [5]:
# FIRST do the first file
address_data = pd.read_csv(files[0], dtype={"LON": float, "LAT": float, "NUMBER": str, "STREET": str, "UNIT": str, 
                            "CITY": str, "DISTRICT": str, "REGION": str, "POSTCODE": str, "ID": str,
                            "HASH": str}, keep_default_na=False)
# combine text LON and LAT fields into a geometry 
geometry = [Point(xy) for xy in zip(address_data.LON, address_data.LAT)]

# add oa prefix to all columns
address_data.columns = ['oa_' + str(col) for col in address_data.columns]
address_data_gd = gpd.GeoDataFrame(address_data, crs=4326, geometry=geometry)

# convert column names to lower case will help later with database queries
address_data_gd.columns = map(str.lower, address_data_gd.columns)

address_data_gd["oa_street_address"] = address_data_gd["oa_number"].map(str) + ' ' +  address_data_gd["oa_street"]

# this is a bit slow, takes about 30-60s
# I don't know why this is necessary, but it is 
address_data_gd['geom'] = address_data_gd['geometry'].apply(lambda x: WKTElement(x.wkt, srid=4326))
address_data_gd.drop('geometry', 1, inplace=True)

In [6]:
table_name = "addresses_table_tx"
#write first 1000 rows to database
address_data_gd.head(1000).to_sql(table_name, engine, if_exists='replace', index=False, 
                                dtype={'geom': Geometry('POINT', srid= 4326)})

In [7]:
chunk_size = 1000
for i in range(1000, address_data_gd.shape[0]+chunk_size, chunk_size):
    address_data_gd[i:(i+chunk_size)].to_sql(table_name, engine, if_exists='append', index=False, 
                                dtype={'geom': Geometry('POINT', srid= 4326)})

In [8]:
# NOW do the rest of the files
for file in files[1:]: # skip the first one since I read it in already
    print(file)
    address_data = pd.read_csv(file, dtype={"LON": float, "LAT": float, "NUMBER": str, "STREET": str, "UNIT": str, 
                                "CITY": str, "DISTRICT": str, "REGION": str, "POSTCODE": str, "ID": str,
                                "HASH": str}, keep_default_na=False)

    # combine text LON and LAT fields into a geometry 
    geometry = [Point(xy) for xy in zip(address_data.LON, address_data.LAT)]
    
    # add oa prefix to all columns
    address_data.columns = ['oa_' + str(col) for col in address_data.columns]
    address_data_gd = gpd.GeoDataFrame(address_data, crs=4326, geometry=geometry)

    # convert column names to lower case will help later with database queries
    address_data_gd.columns = map(str.lower, address_data_gd.columns)
    
    address_data_gd["oa_street_address"] = address_data_gd["oa_number"].map(str) + ' ' +  address_data_gd["oa_street"]
    
    # this is a bit slow, takes about 30-60s
    # I don't know why this is necessary, but it is 
    address_data_gd['geom'] = address_data_gd['geometry'].apply(lambda x: WKTElement(x.wkt, srid=4326))
    address_data_gd.drop('geometry', 1, inplace=True)
    for i in range(0, address_data_gd.shape[0]+chunk_size, chunk_size):
        address_data_gd[i:(i+chunk_size)].to_sql(table_name, engine, if_exists='append', index=False, 
                                                    dtype={'geom': Geometry('POINT', srid= 4326)})
    

/Users/codyschank/Downloads/openaddr-collected-us_south/us/tx/austin.csv
/Users/codyschank/Downloads/openaddr-collected-us_south/us/tx/bandera.csv
/Users/codyschank/Downloads/openaddr-collected-us_south/us/tx/bastrop.csv
/Users/codyschank/Downloads/openaddr-collected-us_south/us/tx/bexar.csv
/Users/codyschank/Downloads/openaddr-collected-us_south/us/tx/brazoria.csv
/Users/codyschank/Downloads/openaddr-collected-us_south/us/tx/brazos.csv
/Users/codyschank/Downloads/openaddr-collected-us_south/us/tx/burleson.csv
/Users/codyschank/Downloads/openaddr-collected-us_south/us/tx/camp.csv
/Users/codyschank/Downloads/openaddr-collected-us_south/us/tx/capcog.csv
/Users/codyschank/Downloads/openaddr-collected-us_south/us/tx/carson.csv
/Users/codyschank/Downloads/openaddr-collected-us_south/us/tx/city_of_abilene.csv
/Users/codyschank/Downloads/openaddr-collected-us_south/us/tx/city_of_amarillo.csv
/Users/codyschank/Downloads/openaddr-collected-us_south/us/tx/city_of_austin.csv
/Users/codyschank/Dow

/Users/codyschank/Downloads/openaddr-collected-us_south/us/tx/shackelford.csv
/Users/codyschank/Downloads/openaddr-collected-us_south/us/tx/statewide-partial.csv
/Users/codyschank/Downloads/openaddr-collected-us_south/us/tx/tarrant-county.csv
/Users/codyschank/Downloads/openaddr-collected-us_south/us/tx/texoma-counties.csv
/Users/codyschank/Downloads/openaddr-collected-us_south/us/tx/titus.csv
/Users/codyschank/Downloads/openaddr-collected-us_south/us/tx/trinity.csv
/Users/codyschank/Downloads/openaddr-collected-us_south/us/tx/uvalde.csv
/Users/codyschank/Downloads/openaddr-collected-us_south/us/tx/vanzandt.csv
/Users/codyschank/Downloads/openaddr-collected-us_south/us/tx/waco.csv
/Users/codyschank/Downloads/openaddr-collected-us_south/us/tx/walker.csv
/Users/codyschank/Downloads/openaddr-collected-us_south/us/tx/washington.csv
/Users/codyschank/Downloads/openaddr-collected-us_south/us/tx/wharton.csv
/Users/codyschank/Downloads/openaddr-collected-us_south/us/tx/williamson.csv
/Users/co

In [ ]:
# do I need 'as dst', or is it just 'dst'
sql_query = """
UPDATE addresses_table_tx dst
SET oa_postcode = src.zcta5ce10
FROM zip5_us src
WHERE ST_Intersects(src.geom, dst.geom);
"""

engine.execute(sql_query)

In [ ]:
# Update street_address column in addresses_table_tx
sql_query = """
UPDATE addresses_table_tx
SET oa_street_address = oa_street_address || ' ' || oa_postcode;
"""
engine.execute(sql_query)

In [9]:
# Delete rows where street number is empty
sql_query = """
DELETE FROM addresses_table_tx
WHERE oa_number = '';
"""
engine.execute(sql_query)

In [10]:
sql_query = """
DELETE FROM addresses_table_tx
WHERE oa_number = '0';
"""
engine.execute(sql_query)

In [11]:
sql_query = """
DELETE FROM addresses_table_tx
WHERE oa_street = '';
"""
engine.execute(sql_query)

In [ ]:
# start here

In [4]:
sql_query = """
UPDATE addresses_table_tx SET oa_street_address = replace(oa_street_address, 'COUNTY ROAD', 'CR');
"""
engine.execute(sql_query)

In [5]:
sql_query = """
UPDATE addresses_table_tx SET oa_street_address = replace(oa_street_address, 'STATE ROAD', 'SR');
"""
engine.execute(sql_query)

In [6]:
# COUNTY RD to CR in openaddress_street_address
sql_query = """
UPDATE addresses_table_tx SET oa_street_address = replace(oa_street_address, 'RANCH ROAD', 'RR');
"""
engine.execute(sql_query)

In [7]:
sql_query = """
UPDATE addresses_table_tx SET oa_street_address = replace(oa_street_address, 'HIGHWAY', 'HWY');
"""
engine.execute(sql_query)

In [8]:
# replace mc c with mcc in openaddress_street_address
sql_query = """
UPDATE addresses_table_tx SET oa_street_address = replace(oa_street_address, 'MC C', 'MCC')
"""
engine.execute(sql_query)

In [9]:
# get rid of double spaces in openaddress_street_address, run twice
sql_query = """
UPDATE addresses_table_tx SET oa_street_address = replace(oa_street_address, '  ', ' ');
"""
engine.execute(sql_query)

In [11]:
# remove duplicates, just take first one
sql_query = """
CREATE TABLE addresses_table_tx_no_dupes AS
SELECT DISTINCT ON (oa_street_address) * FROM addresses_table_tx;
"""
engine.execute(sql_query)

In [13]:
con = None
con = psycopg2.connect(database = dbname, user = username)

# query:
sql_query = """
SELECT * FROM addresses_table_tx_no_dupes LIMIT 1000;
"""
addresses_table_from_sql = pd.read_sql_query(sql_query,con)
addresses_table_from_sql.head(1000)


,oa_lon,oa_lat,oa_number,oa_street,oa_unit,oa_city,oa_district,oa_region,oa_postcode,oa_id,oa_hash,oa_street_address,geom
0,-96.987025,32.677244,1,CANDLER PARK DR,,,,,75052,,625d03c524d2a1db,1 CANDLER PARK DR 75052,0101000020E61000009154956C2B3F58C0ED331DF0AF56...
1,-96.456932,32.834299,1,CANTERBURY CT,,,ROCKWALL,TX,75032,{D2E831DC-C5D0-4DDF-B865-A19F42B9D609}@NCTCOG....,1c6341561d6f61f3,1 CANTERBURY CT 75032,0101000020E6100000E69E64613E1D58C00A4D124BCA6A...
2,-95.628077,29.167185,1,CANTERBURY LN,,,BRAZORIA COUNTY,TX,77486,{44574BBB-FFC5-40A6-9A48-4958CBD94ACF}@H-GAC.TX,09fa4711a3cddd1d,1 CANTERBURY LN 77486,0101000020E61000004B789D6832E857C08BDD3EABCC2A...
3,-96.990787,32.668747,1,CANTERBURY RD,,,,,75052,,8b2a676cdfe3c2b2,1 CANTERBURY RD 75052,0101000020E6100000BED3090D693F58C07C24CA839955...
4,-94.124262,33.438385,1,CANTRELL ST,,,BOWIE,TX,75569,382313,fee85093493b880d,1 CANTRELL ST 75569,0101000020E6100000AD889AE8F38757C009466F021DB8...
5,-101.790365,35.059161,1,CANYON CIR,,LAKE TANGLEWOOD,,,79015,,95563645f1c88243,1 CANYON CIR 79015,0101000020E61000006C9AD255957259C002D946979287...
6,-98.107276,30.021478,1,CANYON CREEK DR,OCCUPIED,,HAYS,TX,78676,,da69b1ad1e91dc40,1 CANYON CREEK DR 78676,0101000020E61000005F0A0F9ADD8658C0DC12149A7F05...
7,-96.849363,33.101633,1,CANYON CREST CT,,,DENTON,TX,75034,{43B8C607-9B9D-48E4-B070-9628680C7185}@NCTCOG....,a23dfaf40addf66a,1 CANYON CREST CT 75034,0101000020E6100000191CCAF55B3658C00B181351028D...
8,-98.956409,31.685912,1,CANYON CRK,,,BROWN,TX,76801,SITE_17038@BROWN.WCTCOG.TX.US,f88d92479c3900a1,1 CANYON CRK 76801,0101000020E6100000C7455ACF35BD58C0946CD0F297AF...
9,-94.811970,32.526305,1,CANYON RIDGE CT,,,,,75604,,42c8c31c7f8d245d,1 CANYON RIDGE CT 75604,0101000020E61000005DF6EB4EF7B357C0941DD1F35D43...
